# Get Coast to Coast Assists and Buckets using PBP data


In [34]:
import pandas as pd
import numpy as np
import time
import datetime
from IPython.display import clear_output
from collections import Counter
from functools import reduce
import itertools
from nba_api.stats.static import teams
from pbpstats.client import Client
from tqdm import tqdm
import os, sys
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

In [35]:
settings = {
    "Games": {"source": "web", "data_provider": "data_nba"},
     "dir": "C:/Users/pansr/Documents/Sra_Coding/NBA/pbpdata",
}
client = Client(settings)
# ID of all games for 2020-21 Season
league = "nba"
season_yr = "2021"
season_type = "Regular Season"
season = client.Season(league, season_yr, season_type)
games_id = []

for final_game in season.games.final_games:
    games_id.append(final_game['game_id'])
print(len(games_id))

1230


In [36]:
settings = {
    "Boxscore": {"source": "file", "data_provider": "data_nba"},
    "Possessions": {"source": "file", "data_provider": "data_nba"},
    "dir": "C:/Users/pansr/Documents/Sra_Coding/NBA/pbpdata"
}
client = Client(settings)
games_list = []
bad_games_list = []
for gameid in tqdm(games_id):
    try:
        games_list.append(client.Game(gameid))
    except:
        # print(gameid)
        bad_games_list.append(gameid)
        continue
print(len(bad_games_list))

100%|██████████| 1230/1230 [00:28<00:00, 43.40it/s]

312


In [25]:
len(bad_games_list)

35

In [26]:
year = int(season_yr)
player_dict = get_players(league = 'NBA', from_year = year, to_year = year)
team_dict = teams.get_teams() # Creating Team Dictionary
team_id = [x['id'] for x in team_dict ]

## Coast to Coast Assists

In [27]:
from pbpstats.resources.enhanced_pbp import FieldGoal
from pbpstats.resources.enhanced_pbp import Rebound
pos_store = []
Rebound_pID = []
Assist_pID = []
i = 0
ii = 0
for game in tqdm(games_list):
    for possession in game.possessions.items:
        for possession_event in possession.events:
            if (isinstance(possession_event, FieldGoal) and possession_event.is_made and possession_event.is_assisted) and (isinstance(possession_event.previous_event, Rebound)) and possession_event.seconds_since_previous_event <= 5:
                if possession_event.previous_event.player1_id == possession_event.player2_id:
                    if not possession_event.previous_event.oreb :
                        pos_store.append(possession)
                        Rebound_pID.append(possession_event.previous_event.player1_id)
                        print ("Rebound Player: {0}".format(possession_event.previous_event.player1_id))
                        Assist_pID.append(possession_event.player2_id)
                        print ("Assist Player: {0}".format(possession_event.player2_id))
                        ii += 1
                        # print("ii = {0}".format(ii))
    i += 1
    # print("i = {0}".format(i))
    clear_output(wait=True)

100%|██████████| 1195/1195 [00:05<00:00, 203.09it/s]


In [28]:
Player_Name = []
a = Assist_pID
Assist_pID1, Assists = np.unique(a, return_counts=True)  
for pID in Assist_pID1:
    Player_Name.append([player['Name'] for player in player_dict if player['pID'] == pID]) 
Player_Name = list(itertools.chain(*Player_Name))

In [29]:
data = pd.DataFrame({'Assist_pID':Assist_pID1,'Player_Name':Player_Name,'Assists':Assists})
data = data.sort_values(by=['Assists'],ascending=False)
data = data.reset_index(drop=True)
data.head()

,Assist_pID,Player_Name,Assists
0,203999,Nikola Jokic,55
1,1627734,Domantas Sabonis,39
2,2544,LeBron James,35
3,201566,Russell Westbrook,33
4,1628404,Josh Hart,33


## Coast to Coast Buckets

In [31]:
pos_store = []
Rebound_pID = []
FGM_pID = []
i = 0
ii = 0
for game in tqdm(games_list):
    for possession in game.possessions.items:
        for possession_event in possession.events:
            if (isinstance(possession_event, FieldGoal) and possession_event.is_made and not possession_event.is_assisted) and (isinstance(possession_event.previous_event, Rebound)) and possession_event.seconds_since_previous_event <= 8:
                if possession_event.previous_event.player1_id == possession_event.player1_id:
                    if not possession_event.previous_event.oreb :
                        pos_store.append(possession)
                        Rebound_pID.append(possession_event.previous_event.player1_id)
                        print ("Rebound Player: {0}".format(possession_event.previous_event.player1_id))
                        FGM_pID.append(possession_event.player1_id)
                        print ("FGM Player: {0}".format(possession_event.player1_id))
                        ii += 1
                        print("ii = {0}".format(ii))
    i += 1
    # print("i = {0}".format(i))
    clear_output(wait=True)

100%|██████████| 1195/1195 [00:06<00:00, 187.86it/s]


In [32]:
Player_Name = []
a = FGM_pID
FGM_pID1, FGM = np.unique(a, return_counts=True)  
for pID in FGM_pID1:
    Player_Name.append([player['Name'] for player in player_dict if player['pID'] == pID]) 
Player_Name = list(itertools.chain(*Player_Name))

In [33]:
data = pd.DataFrame({'FGM_pID':FGM_pID1,'Player_Name':Player_Name,'FGM':FGM})
data = data.sort_values(by=['FGM'],ascending=False)
data = data.reset_index(drop=True)
data.head()

,FGM_pID,Player_Name,FGM
0,203507,Giannis Antetokounmpo,60
1,2544,LeBron James,45
2,1627759,Jaylen Brown,39
3,1630162,Anthony Edwards,35
4,1628983,Shai Gilgeous-Alexander,32
